In [70]:
from abenc_ph_mj18 import PH_ABE, mat_math, Inner_Product
from charm.toolbox.pairinggroup import PairingGroup, ZR, G1, G2, GT, pair
from charm.toolbox.ABEnc import ABEnc

import numpy as np

In [2]:
def ad_auth_setup(n, assump_size, ad, pks):

    pk_ad = []
    k = assump_size
    pk_ad = [[1] * k] * (k+1)

    for i in range(n-1):
        if i + 1 == ad:
            continue

        g_XTA = pks[str(i+1)]['g_1^{X^T A}']

        tmp = []
        for j in range(len(g_XTA)):
            tmp.append([x / y for x,y in zip(pk_ad[j], g_XTA[j])])
        
        pk_ad = tmp

    pk = pks[str(ad)]
    pk['g_1^{X^T A}'] = pk_ad
    
    pks[str(ad)] = pk

    return pks
    

In [3]:
def gen_omega(K, C, n, assump_size):
    H = K['g_2^h']
    k = assump_size
    C_i_s = C['C_i']

    C_ = [1] * (k + 1)
    #C_  = tmp
    for i in range(n-1):
        #print (len(C_), len(C_i_s[i]))
        C_ = [x / y for x,y in zip(C_, C_i_s[i])]

        #print (len(C_))

    return [pair(x,y) for x,y in zip(C_, H)]

In [4]:
n = 10
assump_size = 3
group = PairingGroup('MNT224')
math_lib = mat_math()

ph_abe = PH_ABE(n, assump_size, group, math_lib)

pp, msk = ph_abe.setup()
attributes = Inner_Product(group)
vec_x, vec_v = attributes.gen_x_v(n, assump_size)

pks, sks = ph_abe.auth_setup(pp)

In [5]:
ad = vec_v.index(0) + 1
ad_vec_v = [0] * (n-1) + [1]
print ('Authorized list: ', vec_v)
print ('Adversary:', ad, '\nVector belongs to AD:', ad_vec_v)
pks = ad_auth_setup(n, assump_size, ad, pks)

Authorized list:  [0, 1, 0, 0, 0, 0, 0, 1, 1, 1]
Adversary: 1 
Vector belongs to AD: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [6]:
M = group.random(GT)
print ('M: ', M)

C, vec_s = ph_abe.encrypt(pp, pks, vec_x, M)

M:  [[12496240513970329899101315634405879180464871543696526339821281942514, 7749078562901691341705398472817557017294869198462919018197082321223, 9131301025140182153847485277206680950876510511835747644465852868307], [1102527342141278182746151774926530683988292895672178384278563916987, 567525811742396280087107299731217198741253093157616500567522709452, 4473149641324499460343513315294805417697194562625487924549813912864]]


In [7]:
K = ph_abe.keygen(pp, sks, vec_v)
print ('M_ (Usr):', ph_abe.decrypt(K, C, vec_v,pp))

M_ (Usr): [[12496240513970329899101315634405879180464871543696526339821281942514, 7749078562901691341705398472817557017294869198462919018197082321223, 9131301025140182153847485277206680950876510511835747644465852868307], [1102527342141278182746151774926530683988292895672178384278563916987, 567525811742396280087107299731217198741253093157616500567522709452, 4473149641324499460343513315294805417697194562625487924549813912864]]


In [8]:
K_ = ph_abe.keygen(pp, sks, ad_vec_v)
omega = gen_omega(K_, C, n, assump_size)
M_ = ph_abe.decrypt(K_, C, ad_vec_v, pp)
print ('M_ (Adv):', M_ * math_lib.prod(omega))

M_ (Adv): [[12496240513970329899101315634405879180464871543696526339821281942514, 7749078562901691341705398472817557017294869198462919018197082321223, 9131301025140182153847485277206680950876510511835747644465852868307], [1102527342141278182746151774926530683988292895672178384278563916987, 567525811742396280087107299731217198741253093157616500567522709452, 4473149641324499460343513315294805417697194562625487924549813912864]]


In [69]:
from nizk import NIZK, s_pair

In [66]:
group = PairingGroup('MNT224')
s = group.random(ZR)
e_gh = pair(group.random(G1), group.random(G2))

In [67]:
rps = s_pair(s, e_gh, group)
nizk = NIZK(rps, group)
R, u = nizk.commit("abd")

In [68]:
nizk.verify(R, u, "abd")

True

In [74]:
A = pp['g_1^A'] # global pp
B = pks['1']['g_1^{X^T A}'] #pub key
s = sks['1']['X'] # secret key

In [75]:
s

[[4136790388872615635855264448244061575112554119899701456354709218293,
  14507059771504889314965984071803012873901867223125383517516932224380,
  14693531276219950027469892541795800804837306804290869113407485552337,
  105148006953959492227790301050667398844048146564215649538013153368],
 [12950878762097918449946196538402360015265168640355010166536457547891,
  10655910688793224580297075758723780494050734800716826880605899153884,
  9141297724883498502613695177605617290211858886023640856454809987613,
  7305271810586656616434277444066052224816999907445884371778990310062],
 [9539405781429964499501248862312820024838706771777587651234123384532,
  11189068344071654583473637776075126577721755724198405656289773606996,
  14455150944159860650999907124036411346153619551376590545313810684389,
  14956538391848078349964252122353216923264565649844016055877002828039],
 [5629902237214472450727865127254743554231743606890536904260478038711,
  922610180580166821358236286310983876905775720773387925551691950326

In [78]:
def decompose(s, A, B):
    rps = []
    # check s, A, B same dim
    s_ = np.array(s)
    A_ = np.array(A)
    B_ = np.array(B)

    if s_.shape == A_.shape == B_.shape:

        if s_.ndim == 1:
            zipped = list(zip(s_, A_, B_))
        elif s_.ndim == 2:
            zipped = [list(zip(r1, r2, r3)) for r1, r2,r3 in zip(s_,A_,B_)]

        print (zipped)
    else:
        print (s_.shape, A_.shape, B_.shape)
        print ("Not have same shape")

def same_dim(*args):
    shapes = [np.shape(np.array(arg)) for arg in args]

    return all(shape == shapes[0] for shape in shapes)

In [79]:
decompose(math_lib.transpose(s), A, B)

(4, 4) (3,) (4, 3)
Not have same shape


In [80]:
A

[[6210920301266528640527894807079214500602165062617900333868625936241, 2693704661840088486568063155969700311567336188071870216842793336303],
 [13395093142711244490949414740285904528731771703444461422316169548003, 1386177710749389606492854140816020132082046034321169278703660653055],
 [10479748114164261820989469671675739945182545246576172402210454921050, 12973996679783472661960348458451121808700299006314398530160825904262]]

In [81]:
B

[[[10222292869763003212177921396113081421597626461148751625963526007594, 7501836415110832766526439982389008171919139937759875283451593726244],
  [647476536145263454649663629111386014453753170876950203981922752854, 322556368107729213888535336051555391709953674679240682332598431757],
  [2536847502143815036612177572185957284773069658207229409269250556332, 1367628865557840787406105049478467147576481180683733898159672911230]],
 [[4433493580811008604649168649983764326256759652824948018943987465325, 5239473411124692330600615601504783005173744394501146511311963788531],
  [4597983744197608681959662741795977157917972648565258454052572500285, 5976010693255851213395013403117623666563713170178976011048878178140],
  [1550274815615418836300511053965564998361857714711399563925579774233, 4392228793315853979388053589623079548622285490413501146642170641331]],
 [[7159752001325868940930477425663631876890815739439565026834002960903, 14677864846415100731382352085579691817418720276375753871117815743081],
  [1